In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Enable experimental features
from sklearn.experimental import enable_iterative_imputer  # Это нужно до импорта IterativeImputer

# Preprocessing
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# Column transformation
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV

# Models
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Metrics
from sklearn.metrics import mean_absolute_percentage_error, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, make_scorer

# Encoding (additional)
import category_encoders as ce

import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, PowerTransformer, QuantileTransformer, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from collections import OrderedDict

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingRegressor
cancer = load_breast_cancer()
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.metrics import classification_report, RocCurveDisplay, DetCurveDisplay,PrecisionRecallDisplay
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv'
path_test = 'test.csv'
df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)
print(df.shape, X_test.shape)

In [6]:
# Блок обучения модели

In [ ]:
# Закатегорили 
cat_features = ['Sex', 'Age']
num_features = ['BCS', 'Height', 'Girth', 'Length']

X = df.drop(columns = ['Weight'], axis=1)
y = df['Weight']

num_pipeline = Pipeline(steps=[
    ('poly', PolynomialFeatures(include_bias=False)),  # Полиномиальные признаки
    ('scaler', StandardScaler())  # Стандартизация
])

cat_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

CT = ColumnTransformer([
    ('num', num_pipeline, num_features),  # Числовые данные
    ('cat', cat_pipeline, cat_features)   # Категориальные данные
])

lasso = Pipeline(steps=[
    ('preprocessor', CT),
    ('regressor', Lasso(alpha=0.1, fit_intercept=True, max_iter=500, tol=0.001))
])

svr = Pipeline(steps=[
    ('preprocessor', CT),
    ('regressor', SVR(kernel='linear', C=1, epsilon=0.2))
])

ridge = Pipeline(steps=[
    ('preprocessor', CT),
    ('regressor', Ridge(alpha=0.01, fit_intercept=True, max_iter=500, solver='lsqr', tol=0.001))
])

# Обучение моделей
lasso.fit(X, y)
svr.fit(X, y)
ridge.fit(X, y)

voting_reg = VotingRegressor([("lasso", lasso), ("svr", svr), ("ridge", ridge)])

voting_reg.fit(X, y)

lasso_pred = lasso.predict(X_test)
svr_pred = svr.predict(X_test)
ridge_pred = ridge.predict(X_test)
voting_pred = voting_reg.predict(X_test)

# Вычисление MSE для каждой модели
lasso_mse = mean_squared_error(y_test, lasso_pred)
svr_mse = mean_squared_error(y_test, svr_pred)
ridge_mse = mean_squared_error(y_test, ridge_pred)
voting_mse = mean_squared_error(y_test, voting_pred)

# Вывод MSE для каждой модели
print(f"MSE for Lasso: {lasso_mse:.4f}")
print(f"MSE for SVR: {svr_mse:.4f}")
print(f"MSE for Ridge: {ridge_mse:.4f}")
print(f"MSE for VotingRegressor: {voting_mse:.4f}")

# Визуализация
plt.figure(figsize=(15, 10))
plt.plot(lasso_pred, "gd", label="Lasso")
plt.plot(svr_pred, "b^", label="SVR")
plt.plot(ridge_pred, "ys", label="Ridge")
plt.plot(voting_pred, "r*", ms=10, label="VotingRegressor")
plt.plot(y_test[:20], "ko", label="y_true")

plt.tick_params(axis="x", which="both", bottom=False, top=False, labelbottom=False)
plt.ylabel("Predicted")
plt.xlabel("Training samples")
plt.legend(loc="best")
plt.title("Regressor predictions and their average")

plt.show()


In [7]:
# Блок предсказания с использованием тестового набора

In [ ]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict = voting_reg.predict(X_test)
y_predict.shape